# Aggregating

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/dc-wikia-data-clean.csv')
df.head()

,page_id,name,urlslug,id,align,eye,hair,sex,gsm,alive,appearances,first appearance,year
0,1422,Batman (Bruce Wayne),\/wiki\/Batman_(Bruce_Wayne),Secret,Good,Blue,Black,Male,NaN,True,3093.0,"1939, May",1939.0
1,23387,Superman (Clark Kent),\/wiki\/Superman_(Clark_Kent),Secret,Good,Blue,Black,Male,NaN,True,2496.0,"1986, October",1986.0
2,1458,Green Lantern (Hal Jordan),\/wiki\/Green_Lantern_(Hal_Jordan),Secret,Good,Brown,Brown,Male,NaN,True,1565.0,"1959, October",1959.0
3,1659,James Gordon (New Earth),\/wiki\/James_Gordon_(New_Earth),Public,Good,Brown,White,Male,NaN,True,1316.0,"1987, February",1987.0
4,1576,Richard Grayson (New Earth),\/wiki\/Richard_Grayson_(New_Earth),Secret,Good,Blue,Black,Male,NaN,True,1237.0,"1940, April",1940.0


## Basic summarising and descriptive statistics

In [3]:
df.mean()

page_id        147441.209252
alive               0.754060
appearances        23.625134
year             1989.766662
dtype: float64

In [7]:
df.max()

page_id                                404010
name                   Zzlrrrzzzm (New Earth)
urlslug        \/wiki\/Zzlrrrzzzm_(New_Earth)
alive                                    True
appearances                              3093
year                                     2013
dtype: object

In [8]:
df['sex'].unique()

array(['Male', 'Female', nan, 'Genderless', 'Transgender'], dtype=object)

In [13]:
len(df[df['sex'].isnull()])

125

In [9]:
df['sex'].value_counts()

Male           4783
Female         1967
Genderless       20
Transgender       1
Name: sex, dtype: int64

In [ ]:
df['year'].min()

In [ ]:
df['year'].max()

## `groupby`

**Figure copied from [Jake Vanderplas's book](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.08-Aggregation-and-Grouping.ipynb).**

![title](figures/jake-vanderplas-split-apply-combine.png)

### Basic built-in aggregation functions

`count`, `sum`, `mean`, `median`, `std`, `var`, `min`, `max`, `prod`, `first`, `last`.

In [15]:
df.groupby('sex').count()

,page_id,name,urlslug,id,align,eye,hair,gsm,alive,appearances,first appearance,year
sex,,,,,,,,,,,,
Female,1967,1967,1967,1390,1747,1130,1621,29,1967,1880,1947,1947
Genderless,20,20,20,16,20,8,3,1,20,19,20,20
Male,4783,4783,4783,3422,4427,2108,2984,34,4783,4527,4735,4735
Transgender,1,1,1,0,1,0,0,0,1,1,1,1


In [16]:
df.groupby('sex').mean()

,page_id,alive,appearances,year
sex,,,,
Female,159307.531774,0.800203,22.484574,1992.621983
Genderless,132061.750000,0.750000,12.842105,1990.850000
Male,141814.427138,0.734058,24.499890,1988.532841
Transgender,317067.000000,0.000000,4.000000,2009.000000


With multiple level (search for `MultiIndex` for more info).

In [17]:
df.groupby(['sex', 'align']).count()

page_id  name  urlslug    id  eye  hair  gsm  alive  \
sex         align                                                           
Female      Bad           597   597      597   419  325   472    7    597   
            Good          953   953      953   714  582   812   18    953   
            Neutral       196   196      196   138  125   169    3    196   
            Reformed        1     1        1     0    1     1    0      1   
Genderless  Bad            11    11       11     9    3     1    0     11   
            Good            6     6        6     4    2     2    0      6   
            Neutral         3     3        3     3    3     0    1      3   
Male        Bad          2223  2223     2223  1542  861  1204    8   2223   
            Good         1843  1843     1843  1419  919  1306   25   1843   
            Neutral       359   359      359   254  191   253    1    359   
            Reformed        2     2        2     2    1     2    0      2   
Transgender Bad             1     1        1     0    0     0    0      1   

                      appearances  first appearance  year  
sex         align                                          
Female      Bad               568               596   596  
            Good              909               941   941  
            Neutral           193               194   194  
            Reformed            1                 1     1  
Genderless  Bad                10                11    11  
            Good                6                 6     6  
            Neutral             3                 3     3  
Male        Bad              2088              2208  2208  
            Good             1756              1819  1819  
            Neutral           348               353   353  
            Reformed            2                 2     2  
Transgender Bad                 1                 1     1

### Custom aggregations

Specifying pandas built-in functions by name.

In [19]:
df.groupby('sex').agg({'page_id': 'count', 'appearances': 'mean'})

,page_id,appearances
sex,,
Female,1967,22.484574
Genderless,20,12.842105
Male,4783,24.499890
Transgender,1,4.000000


Using multiple functions for the same column.

In [20]:
df.groupby('sex').agg({'appearances': ['mean', 'std']})

appearances           
                   mean        std
sex                               
Female        22.484574  68.717080
Genderless    12.842105  11.922263
Male          24.499890  95.168200
Transgender    4.000000        NaN

Using custom python functions.

In [30]:
def values_range(x):
    return max(x) - min(x)

In [31]:
df.groupby('sex').agg({'appearances': values_range})

,appearances
sex,
Female,1230.0
Genderless,35.0
Male,3092.0
Transgender,0.0


# Pivot tables

In [32]:
df.pivot_table(index='sex', columns='gsm', values='appearances')

gsm,Bisexual,Homosexual
sex,,
Female,64.4,47.291667
Genderless,20.0,NaN
Male,109.0,33.366667


## *Exercise*

Among bisexual characters, what is the sex that appears the most (on average)? Is that the same for homosexual characters?